In [1]:
using DataStructures
using FileIO
using Meshes
using MeshIO
using Makie
using GLMakie
using GeometryBasics
using ProgressBars

In [2]:
include("contractions.jl")
include("visualisation.jl")
include("preprocessing.jl")

loadsc (generic function with 1 method)

In [76]:
bunidata = loadsc("ply/bunny.ply")

# rotate by 90 degrees around the x-axis
println("Get rotated lmao")
for vertex in bunidata.coords
    vertex[2], vertex[3] = vertex[2] * cos(π / 2) - vertex[3] * sin(π / 2), vertex[2] * sin(π / 2) + vertex[3] * cos(π / 2)
end

buni = initialContractedSimplicialComplex2D(bunidata)
butthole = getborder(buni.contracted)
# convert Set{Tuple{Int64, Int64}}) to Array{GeometryBasics.Point{3,Float32}})
butthole_array = collect(butthole)
butthole = [GeometryBasics.Point3(buni.contracted.coords[butthole_array[i][1]]...) for i in 1:length(butthole_array)]
fillholes!(buni.contracted)

0.0%┣                                            ┫ 0/69.5k [00:00<00:-1, -0s/it]
2.9%┣█                                     ┫ 2.0k/69.5k [00:00<00:02, 39.8kit/s]
6.2%┣██▍                                   ┫ 4.3k/69.5k [00:00<00:02, 42.6kit/s]
9.7%┣███▊                                  ┫ 6.7k/69.5k [00:00<00:01, 44.4kit/s]
13.1%┣████▉                                ┫ 9.1k/69.5k [00:00<00:01, 45.1kit/s]
16.4%┣██████                              ┫ 11.4k/69.5k [00:00<00:01, 45.4kit/s]
18.8%┣██████▊                             ┫ 13.0k/69.5k [00:00<00:01, 43.1kit/s]
21.3%┣███████▋                            ┫ 14.8k/69.5k [00:00<00:01, 41.9kit/s]
24.9%┣█████████                           ┫ 17.3k/69.5k [00:00<00:01, 43.0kit/s]
28.3%┣██████████▏                         ┫ 19.7k/69.5k [00:00<00:01, 43.4kit/s]
32.1%┣███████████▋                        ┫ 22.3k/69.5k [00:01<00:01, 44.3kit/s]
35.3%┣████████████▊                       ┫ 24.6k/69.5k [00:01<00:01, 44.3kit/s]
39.1%┣██████████████        

Get rotated lmao


99.4%┣███████████████████████████████████▊┫ 69.0k/69.5k [00:02<00:00, 40.1kit/s]
100.0%┣███████████████████████████████████┫ 69.5k/69.5k [00:02<00:00, 40.1kit/s]
100.0%┣███████████████████████████████████┫ 69.5k/69.5k [00:02<00:00, 40.1kit/s]


Finding buttholes
Buttholes left: 223
Buttholes left: 184
Buttholes left: 142
Buttholes left: 62
Buttholes left: 22
Constipating 5 buttholes


0.0%┣                                                ┫ 0/5 [00:00<00:00, -0s/it]
100.0%┣████████████████████████████████████████████┫ 5/5 [00:00<00:00, 2.2kit/s]
100.0%┣████████████████████████████████████████████┫ 5/5 [00:00<00:00, 2.0kit/s]


In [77]:
println("Calculating fundamental quadratics")
calculateFundamentalQuadratics!(buni)

println("Constructing initial priority queue")
pq = PriorityQueue()
for (edge, triangle) in ProgressBar(buni.contracted._edge_to_triangles)
    pq[edge] = error(buni, edge)
end

Calculating fundamental quadratics
Constructing initial priority queue


0.0%┣                                           ┫ 0/104.5k [00:00<00:-2, -0s/it]
3.1%┣█▏                                   ┫ 3.3k/104.5k [00:00<00:02, 65.3kit/s]
6.4%┣██▍                                  ┫ 6.7k/104.5k [00:00<00:01, 66.2kit/s]
9.6%┣███▌                                ┫ 10.0k/104.5k [00:00<00:01, 66.2kit/s]
12.6%┣████▍                              ┫ 13.2k/104.5k [00:00<00:01, 65.3kit/s]
15.6%┣█████▌                             ┫ 16.3k/104.5k [00:00<00:01, 65.0kit/s]
18.7%┣██████▌                            ┫ 19.5k/104.5k [00:00<00:01, 64.6kit/s]
21.9%┣███████▋                           ┫ 22.9k/104.5k [00:00<00:01, 64.9kit/s]
25.1%┣████████▉                          ┫ 26.2k/104.5k [00:00<00:01, 65.1kit/s]
28.2%┣█████████▉                         ┫ 29.5k/104.5k [00:00<00:01, 65.1kit/s]
31.2%┣███████████                        ┫ 32.7k/104.5k [00:01<00:01, 64.9kit/s]
34.2%┣████████████                       ┫ 35.7k/104.5k [00:01<00:01, 64.5kit/s]
37.3%┣█████████████         

In [ ]:
i = 0
while i < 7486 # problematic edge: (2060, 2061)
    global myedge
    try
        global myedge
        myedge = dequeue!(pq)
        while !issafe(buni.contracted, myedge)
            myedge = dequeue!(pq)
        end
    catch e
        break
    end

    new_vertex = contract!(buni, myedge)
    for edge in buni.contracted._vertex_to_edges[new_vertex]
        pq[edge] = error(buni, edge)
    end
    i += 1
end
println(i)

7486


In [79]:
i

7486

In [80]:
buni.contracted.coords[new_vertex]

3-element Vector{Float32}:
 -0.07101557
  0.0450611
  0.156148

In [81]:
include("visualisation.jl")
visualize(buni.contracted)

GLMakie.Screen(...)

In [83]:
myedge

(2060, 2061)